In [ ]:
import torch
import torch.nn as nn

In [ ]:
"""
NLP中的输入总是有点混，维度是怎么来的，经过embedding的一个句子是：
[[0.1, 0.2, 0.3], [0.55, 0.44, 0.44]]
多个句子是:
[[[0.1, 0.2, 0.3], [0.55, 0.44, 0.44]], 
[[0.1, 0.2, 0.3], [0.55, 0.44, 0.44]]]
"""
input = torch.randn(6, 3, 10)
"""
data.shape = [batch_size, sequence_length, vocab_dim]还是[sequence_length, batch_size, vocab_dim]
6是sample数
3是每个句子的长度
10是每个单词的embedding维度
"""
print(input)

In [ ]:
"""
这些构造都很有意思，感觉是先
rnn = nn.RNNCell(10, 20) - 构造结构
hx = rnn(input[i], hx) - 真正的计算
"""
rnn = nn.RNNCell(10, 20)
hx = torch.randn(3, 20)
output = []
for i in range(6):
    hx = rnn(input[i], hx)
    output.append(hx)
print(output)